In [0]:
from pyspark.sql.functions import col, sum

In [0]:
customers_df = spark.read.table("hive_metastore.default.fake_customers")
orders_df = spark.read.table("hive_metastore.default.fake_orders")
zipcodes_df = spark.read.csv("/Volumes/mp_spark_deep_ddve/default/zipcode_data/zipcode_data.csv", header=True, inferSchema=True)

In [0]:
joined_df = customers_df.hint("sort_merge_join").join(orders_df, customers_df.customer_id == orders_df.customer_id, how="left")
joined_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   SortMergeJoin [customer_id#803L], [cast(customer_id#833 as bigint)], LeftOuter
   :- Sort [customer_id#803L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(customer_id#803L, 200), ENSURE_REQUIREMENTS, [plan_id=794]
   :     +- Project [customer_id#803L, name#804, email#805, addressLine1#806, addressLine2#807, city#808, state#809, zip_code#810, phone_number#811]
   :        +- Filter if (isnotnull(_databricks_internal_edge_computed_column_skip_row#982)) (_databricks_internal_edge_computed_column_skip_row#982 = false) else isnotnull(raise_error(DELTA_SKIP_ROW_COLUMN_NOT_FILLED, map(keys: [], values: []), NullType))
   :           +- FileScan parquet hive_metastore.default.fake_customers[customer_id#803L,name#804,email#805,addressLine1#806,addressLine2#807,city#808,state#809,zip_code#810,phone_number#811,_databricks_internal_edge_computed_column_skip_row#982] Batched: true, DataFilters: [], Form

In [0]:
agg_zipcodes_df = zipcodes_df.groupBy("zipCode").agg(sum("population").alias("total_population"))
joined_zipcodes_df = joined_df.hint("sort_merge_join").join(agg_zipcodes_df, joined_df.zip_code == agg_zipcodes_df.zipCode, how="left")

In [0]:
joined_zipcodes_df.show(1000)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can